In [1]:
import pandas as pd
import re

# Read your CSV file and the Excel file
df_services = pd.read_csv('D:\\DataFrame\\Services.csv', header=0, encoding='ISO-8859-1')
df_medicine = pd.read_csv('D:\\DataFrame\\Medicine.csv', usecols=['ItemName'], encoding='ISO-8859-1')
df_plan_of_care = pd.read_excel("Downloads//EMR.xlsx")

# Filter the DataFrame to keep only rows where the 'Template' column is 'Plan of care'
df_filtered = df_plan_of_care[df_plan_of_care['Template'] == 'Plan of care']

# Split the ServiceName into two columns based on the first occurrence of '('
df_services[['ServiceName_Part1', 'ServiceName_Part2']] = df_services['ServiceName'].str.split('(', n=1, expand=True)

# Clean the second part of the ServiceName
df_services['ServiceName_Part2'] = df_services['ServiceName_Part2'].str.strip().str.rstrip(')').str.lstrip('(')
df_services = df_services[['ServiceName_Part1', 'ServiceName_Part2']]

# Create a new column with the first string from ItemName
df_medicine['FirstString'] = df_medicine['ItemName'].apply(lambda x: x.split()[0])

# List of tests and medicine keywords (replace the ellipsis with actual lists)
tests = [
    "PT INR", "COMPLETE HGM", "ESR", "PERIPHERAL BLOOD FILM", "HBA1C", "APTT", "TSH", 
    "RA FACTOR", "ECG", "BLOOD C/S", "NT PROBNP", "PROCALCITONIN", "URINE R/M C/S", 
    "LFT", "KFT", "MRI BRAIN WITH MRA BRAIN NECK", "Calcium", "Urine R/M", "ECHO", 
    "CPK", "CKMB", "TROP I", "Chest X-ray PA", "B12", "USG Whole Abdomen", 
    "ANA - IF", "Lipid Profile", "Anti TPO", "USG Neck", "Thyroid Profile", "MRI Brain", 
    "Whole Spine Screening", "CT Chest", "ANA Immunoblot 17 Antigen", "DEXA Scan Whole Body", 
    "Sudoscan", "BMI", "Renal Artery Doppler", "Holter", "FBS", "PPBS", 
    "Sugar Charting with Glucometer", "Fundus and Refraction", "CBC", "RFT", "INR", 
    "AFP", "PSA", "Urine Routine and Culture", "USG Abdomen with PVR", "Uroflowmetry", 
    "Ultrasound Thyroid Gland", "HbsAg", "Anti HCV", "NT Pro BNP", "Peripheral Smear for Blood Picture", 
    "Dietician Opinion for Diabetic Diet", "UGIE/RUT", "Fibroscan-Liver", 
    "CECT Abdomen (Negative Oral Contrast)", "CECT Abdomen (IV and Oral Contrast)", "PS MP", 
    "Malarial Antigen", "Dengue NS1 Antigen", "Dengue Serology IgG IgM", "Scrub Typhus IgM", 
    "Typhi Dot IgM", "Chikungunya Serology IgG IgM", "Leptospira Serology IgG IgM", "Blood Sugar R", 
    "ABPM", "USG W/A", "SR PSA Free Total", "Chest X-Ray PA", "Sputum AFB Gram Stain C/S KOH", 
    "GeneXpert", "MRI Angio Brain and Neck", "Ultrasound Abdomen", "FibroScan", "PT INR APTT", 
    "Blood C/S"
]

medicine_keywords = [
    'CAPSULES', 'CREAM & OINTMENT', 'DROPS', 'GARGLE', 'INHALER',
    'INJECTION', 'LOTIONS', 'MISCELLANEOUS', 'PATCH', 'POWDER',
    'RESPULES', 'ROTACAP', 'SACHET', 'SPRAYS', 'SUPPOSITORY', 'SYRUP',
    'TABLET', 'VACCINE', 'CREAM', 'EYE OINTMENT', 'GEL',
    'SKIN OINTMENT', 'EAR DROP', 'EYE & EAR DROP', 'EYE DROP',
    'NASAL DROP', 'ORAL DROP', 'NASAL SPRAY', 'SKIN SPRAY', 'SPACER', 'TAB',
    'CustomMedication', 'SOLUTION', 'TOOTH PASTE', 'TAB.', 'tab', 'Tab.', 
    'Tab', 'SYP', 'syp', 'syp.', 'Syp', 'Syp.', 'INJ', 'INJ.', 'inj', 
    'Inj.', 'Inj', 'CAP', 'CAP.', 'cap.', 'cap', 'Cap', 'Cap.', 'NS', 'NS.'
]

# Iterate over each row in the filtered DataFrame
for _, row in df_filtered.iterrows():
    visit_id = row['VisitId']
    data = row['Data']
    
    # Split the 'Data' column into sentences based on '<br/>' or ',' as delimiters
    sentences = [sentence.strip() for sentence in data.replace('<br/>', ',').split(',')]
    
    matching_tests = set()
    matching_medicine = set()
    
    for sentence in sentences:
        lower_sentence = sentence.lower()
        lower_service_part1 = df_services['ServiceName_Part1'].str.lower()
        lower_service_part2 = df_services['ServiceName_Part2'].str.lower()
        
        if lower_sentence in lower_service_part1.values or lower_sentence in lower_service_part2.values:
            matching_tests.add(sentence.upper())  # Convert to uppercase before adding

        # Find matches in the test list
        matches = [test.lower() for test in tests if re.search(rf'\b{re.escape(test)}\b', sentence, re.IGNORECASE)]
        for match in matches:
            matching_tests.add(match.upper())  # Convert to uppercase before adding
            
        # Find matches in the medicine keywords
        for medicine in medicine_keywords:
            if re.search(rf'\b{re.escape(medicine.lower())}\b', lower_sentence):
                matching_medicine.add(sentence.upper())  # Add the original sentence in uppercase
    
    # Print the matching tests and medicines with their VisitId
    if matching_tests or matching_medicine:
        print(f"\nVisitId: {visit_id}")
        if matching_tests:
            print("Matching Tests : ==> ")
            for match in matching_tests:
                print(match)
        if matching_medicine:
            print("Matching Medicines : ==> ")
            for match in matching_medicine:
                print(match)



VisitId: 2379379
Matching Tests : ==> 
CBC
KFT
LFT
Matching Medicines : ==> 
TAB CALPOL 500 MG SOS FOR PAIN

VisitId: 2379393
Matching Medicines : ==> 
TAB. IVABRAD 5 MG 1 TAB TWICE DAILY (AT 7 AM AND 7 PM)
TAB. PANTOCID 40 MG ORALLYONCE DAILY BEFORE BREAKFAST
TAB. MUCINAC 600 MG 1 TAB TWICE DAILY DISSOLVE IN WATER
TAB. PLANEP T 25/10 MG 1 TAB ONCE DAILY (AT 10 AM )
TAB. DAPLO 10 MG 1 TAB ONCE DAILY WITH LUNCH
TAB. BRILINTA 90 MG ORALLY TWICE DAILY (10:00 AM / 10:00 PM)
TAB. ATORVA 80 MG ORALLYONCE DAILY AT BED TIME
TAB. ECOSPRIN 75 MG ORALLYONCE DAILY AT BED TIME
TAB. AZMARDA 100 MG 1 TAB TWICE DAILY (AT 7 AMD AND 7 PM)

VisitId: 2379393
Matching Medicines : ==> 
LOFTAIR INHALER INHALATION ONCE DAILY FOLLOWED BY GARGLES

VisitId: 2379395
Matching Tests : ==> 
ECHO
FIBROSCAN
ECG
Matching Medicines : ==> 
TAB JARDIANCE MET 12.5/500 MG 1 BD
TAB THYROX 37.5 MCG 1 OD
TAB ROSEDAY 10 MG 1 OD
TAB SHALCAL XT 1 OD

VisitId: 2379412
Matching Tests : ==> 
PSA
FIBROSCAN
Matching Medicines : ==> 


In [28]:
import pandas as pd
import re

# Read your CSV file and the Excel file
df_services = pd.read_csv('D:\\DataFrame\\Services.csv', header=0, encoding='ISO-8859-1')
df_medicine = pd.read_csv('D:\\DataFrame\\Medicine.csv', usecols=['ItemName'], encoding='ISO-8859-1')
df_plan_of_care = pd.read_excel("Downloads//EMR.xlsx")

# Filter the DataFrame to keep only rows where the 'Template' column is 'Plan of care'
df_filtered = df_plan_of_care[df_plan_of_care['Template'] == 'Plan of care']

# Split the ServiceName into two columns based on the first occurrence of '('
df_services[['ServiceName_Part1', 'ServiceName_Part2']] = df_services['ServiceName'].str.split('(', n=1, expand=True)

# Clean the second part of the ServiceName
df_services['ServiceName_Part2'] = df_services['ServiceName_Part2'].str.strip().str.rstrip(')').str.lstrip('(')
df_services = df_services[['ServiceName_Part1', 'ServiceName_Part2']]

# Create a new column with the first string from ItemName
df_medicine['FirstString'] = df_medicine['ItemName'].apply(lambda x: x.split()[0])

# Lists of tests and medicine keywords
tests = [
    "PT INR", "COMPLETE HGM", "ESR", "PERIPHERAL BLOOD FILM", "HBA1C", "APTT", "TSH", 
    "RA FACTOR", "ECG", "BLOOD C/S", "NT PROBNP", "PROCALCITONIN", "URINE R/M C/S", 
    "LFT", "KFT", "MRI BRAIN WITH MRA BRAIN NECK", "Calcium", "Urine R/M", "ECHO", 
    "CPK", "CKMB", "TROP I", "Chest X-ray PA", "B12", "USG Whole Abdomen", 
    "ANA - IF", "Lipid Profile", "Anti TPO", "USG Neck", "Thyroid Profile", "MRI Brain", 
    "Whole Spine Screening", "CT Chest", "ANA Immunoblot 17 Antigen", "DEXA Scan Whole Body", 
    "Sudoscan", "BMI", "Renal Artery Doppler", "Holter", "FBS", "PPBS", 
    "Sugar Charting with Glucometer", "Fundus and Refraction", "CBC", "RFT", "INR", 
    "AFP", "PSA", "Urine Routine and Culture", "USG Abdomen with PVR", "Uroflowmetry", 
    "Ultrasound Thyroid Gland", "HbsAg", "Anti HCV", "NT Pro BNP", "Peripheral Smear for Blood Picture", 
    "Dietician Opinion for Diabetic Diet", "UGIE/RUT", "Fibroscan-Liver", 
    "CECT Abdomen (Negative Oral Contrast)", "CECT Abdomen (IV and Oral Contrast)", "PS MP", 
    "Malarial Antigen", "Dengue NS1 Antigen", "Dengue Serology IgG IgM", "Scrub Typhus IgM", 
    "Typhi Dot IgM", "Chikungunya Serology IgG IgM", "Leptospira Serology IgG IgM", "Blood Sugar R", 
    "ABPM", "USG W/A", "SR PSA Free Total", "Chest X-Ray PA", "Sputum AFB Gram Stain C/S KOH", 
    "GeneXpert", "MRI Angio Brain and Neck", "Ultrasound Abdomen", "FibroScan", "PT INR APTT", 
    "Blood C/S"
]

medicine_keywords = [
    'CAPSULES', 'CREAM & OINTMENT', 'DROPS', 'GARGLE', 'INHALER',
    'INJECTION', 'LOTIONS', 'MISCELLANEOUS', 'PATCH', 'POWDER',
    'RESPULES', 'ROTACAP', 'SACHET', 'SPRAYS', 'SUPPOSITORY', 'SYRUP',
    'TABLET', 'VACCINE', 'CREAM', 'EYE OINTMENT', 'GEL',
    'SKIN OINTMENT', 'EAR DROP', 'EYE & EAR DROP', 'EYE DROP',
    'NASAL DROP', 'ORAL DROP', 'NASAL SPRAY', 'SKIN SPRAY', 'SPACER', 'TAB',
    'CustomMedication', 'SOLUTION', 'TOOTH PASTE', 'TAB.', 'tab', 'Tab.', 
    'Tab', 'SYP', 'syp', 'syp.', 'Syp', 'Syp.', 'INJ', 'INJ.', 'inj', 
    'Inj.', 'Inj', 'CAP', 'CAP.', 'cap.', 'cap', 'Cap', 'Cap.', 'NS', 'NS.'
]

# Initialize an empty list to store the output rows
output_rows = []

# Iterate over each row in the filtered DataFrame
for _, row in df_filtered.iterrows():
    visit_id = row['VisitId']
    data = row['Data']
    
    # Split the 'Data' column into sentences based on '<br/>' or ',' as delimiters
    sentences = [sentence.strip() for sentence in data.replace('<br/>', ',').split(',')]
    
    matching_tests = set()
    matching_medicine = set()
    
    for sentence in sentences:
        lower_sentence = sentence.lower()
        lower_service_part1 = df_services['ServiceName_Part1'].str.lower()
        lower_service_part2 = df_services['ServiceName_Part2'].str.lower()
        
        if lower_sentence in lower_service_part1.values or lower_sentence in lower_service_part2.values:
            matching_tests.add(sentence.upper())  # Convert to uppercase before adding

        # Find matches in the test list
        matches = [test.lower() for test in tests if re.search(rf'\b{re.escape(test)}\b', sentence, re.IGNORECASE)]
        for match in matches:
            matching_tests.add(match.upper())  # Convert to uppercase before adding
            
        # Find matches in the medicine keywords
        for medicine in medicine_keywords:
            if re.search(rf'\b{re.escape(medicine.lower())}\b', lower_sentence):
                matching_medicine.add(sentence.upper())  # Add the original sentence in uppercase
    
    # If there are matches, store them in the output rows list
    if matching_tests or matching_medicine:
        output_rows.append({
            'VisitId': visit_id,
            'Matching Medicines': ', '.join(matching_medicine),
            'Matching Tests': ', '.join(matching_tests)
        })

# Convert the output rows to a DataFrame
df_output = pd.DataFrame(output_rows)

# Save the output to a CSV file
df_output.to_csv('D:\\DataFrame\\matched_output.csv', index=False)

print("Output saved to 'D:\\DataFrame\\matched_output.csv'")


Output saved to 'D:\DataFrame\matched_output.csv'
